In [26]:
import pandas as pd

cgv = pd.read_csv('../cgv_crawling.csv')
daum = pd.read_csv('../daum_crawling.csv')
lotte = pd.read_csv('../lotte_crawling.csv')
mega = pd.read_csv('../mega_crawling.csv')
# naver = pd.read_csv('ddd.csv')

#* 필요시 사용
# # 네이버 테이블 분리
# naver_review = naver.loc[:,['name', 'review']]
# naver.drop(columns='review', inplace=True)

In [14]:
# 제목 전처리
lotte.name = lotte.name.apply(lambda x: ''.join(x.split()))
mega.name = mega.name.apply(lambda x: ''.join(x.split()))
cgv.name = cgv.name.apply(lambda x: ''.join(x.split(',')))

Daum 영화 기준 // 같은 영화제목만 추출

In [27]:
# 다음 기준으로 제목 동일한 데이터만 추출
column = list(daum.name.values)

cgv = cgv[cgv['name'].isin(column)]
mega = mega[mega['name'].isin(column)]
lotte = lotte[lotte['name'].isin(column)]

#print(len(cgv.name), len(mega.name), len(lotte.name))

In [16]:
# 리뷰 데이터 합치기
def merge(df1, df2):
    merge_df = pd.merge(df1, df2, on='name', suffixes=('_df1', '_df2'))
    merge_df['review'] = merge_df['review_df1'] + merge_df['review_df2']
    merge_df = merge_df.drop(['review_df1', 'review_df2'], axis=1)
    
    return merge_df

result1 = merge(lotte, cgv)
result2 = merge(result1, mega)
result3 = merge(result2, daum)
# result = merge(result3, naver_review)
# len(result3.iloc[1,:].review)

리뷰 통합 // review.csv

In [18]:
result3.to_csv('review.csv', index=False)

모델 수행

In [19]:
import pandas as pd
import re

df = pd.read_csv('review.csv')

def preprocessing(review):
    filtered_review = re.sub("[^a-zA-Zㄱ-ㅎ가-힣 '']", "", review)
    return filtered_review

df['review'] = df['review'].apply(lambda x: [preprocessing(x)])

특수문자 전처리 된 리뷰  // reciew_clean.csv

In [21]:
df.to_csv('review_clean.csv', index=False)

감정분석 수행

In [28]:
import pandas as pd
from model import process_news_keyword

df = pd.read_csv('review_clean.csv')

result = process_news_keyword(df)
result